# Importing and Exporting Data

Data can be imported into Google BigQuery from a CSV file stored within Google Cloud Storage, or it can be streamed directly into BigQuery from Python code.

Similarly, the results of a query can be exported to Google Cloud Storage as a set of shards, or they can be streamed directly into a file within Datalab. Note that for larger data sizes, it is recommended to choose the sharded method.

In [1]:
from google.datalab import Context
import google.datalab.bigquery as bq
import google.datalab.storage as storage
import pandas as pd
try:
  from StringIO import StringIO
except ImportError:
  from io import BytesIO as StringIO

# Importing Data

The first step to analyzing and querying your data is importing it. For this demo, we'll create a temporary table in a temporary dataset within BigQuery, using a small data file within Cloud Storage.

## Importing Data from Cloud Storage

To interact with Google Cloud Storage, Datalab includes the `%%gcs` command. First, see the available options on `%%gcs`:

In [2]:
%gcs -h

usage: %gcs [-h] {copy,create,delete,list,read,view,write} ...

Execute various Google Cloud Storage related operations. Use "%gcs <command>
-h" for help on a specific command.

positional arguments:
  {copy,create,delete,list,read,view,write}
                        commands
    copy                Copy one or more Google Cloud Storage objects to a
                        different location.
    create              Create one or more Google Cloud Storage buckets.
    delete              Delete one or more Google Cloud Storage buckets or
                        objects.
    list                List buckets in a project, or contents of a bucket.
    read                Read the contents of a Google Cloud Storage object
                        into a Python variable.
    view                View the contents of a Google Cloud Storage object.
    write               Write the value of a Python variable to a Google Cloud
                        Storage object.

optional arguments:
  -h, --

Let's use the `read` option to read a storage object into a local Python variable:

In [3]:
%%gcs read --object gs://cloud-datalab-samples/cars.csv --variable cars

In [4]:
print(cars)

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000.00
1999,Chevy,"Venture Extended Edition","",4900.00
1999,Chevy,"Venture Extended Edition",Very Large,5000.00
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.00



In [5]:
# Create the schema, conveniently using a DataFrame example.
df = pd.read_csv(StringIO(cars))
schema = bq.Schema.from_data(df)

# Create the dataset
bq.Dataset('importingsample').create()

# Create the table
sample_table = bq.Table('importingsample.cars').create(schema = schema, overwrite = True)

In [8]:
df.shape

(4, 5)

In [9]:
sample_table.load('gs://cloud-datalab-samples/cars.csv', mode='append',
                  source_format = 'csv', csv_options=bq.CSVOptions(skip_leading_rows = 1))

Job my-udemy-project-210404/job_h7g-LHu9DI7wf7I1ngKZ78XBsUFp completed

In [10]:
%%bq query -n importingSample
SELECT * FROM importingsample.cars

In [11]:
%bq execute -q importingSample

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000.0
1999,Chevy,Venture Extended Edition,,4900.0
1999,Chevy,Venture Extended Edition,Very Large,5000.0
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.0


## Importing Data from a DataFrame

In [12]:
cars2 = storage.Object('cloud-datalab-samples', 'cars2.csv').read_stream()
df2 = pd.read_csv(StringIO(cars2))
df2

,Year,Make,Model,Description,Price
0,2010,Honda,Civic,NaN,15000.0
1,2015,Tesla,Model S,NaN,64900.0


In [13]:
df2.fillna(value='', inplace=True)
df2

,Year,Make,Model,Description,Price
0,2010,Honda,Civic,,15000.0
1,2015,Tesla,Model S,,64900.0


In [15]:
sample_table.insert(df2)
sample_table.to_dataframe()

,Year,Make,Model,Description,Price
0,1997,Ford,E350,"ac, abs, moon",3000.0
1,1999,Chevy,Venture Extended Edition,,4900.0
2,1999,Chevy,Venture Extended Edition,Very Large,5000.0
3,1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.0
4,2010,Honda,Civic,,15000.0
5,2015,Tesla,Model S,,64900.0


# Exporting Data

## Exporting Data to Cloud Storage

In [16]:
project = Context.default().project_id
sample_bucket_name = project + '-datalab-samples'
sample_bucket_path = 'gs://' + sample_bucket_name
sample_bucket_object = sample_bucket_path + '/tmp/cars.csv'
print('Bucket: ' + sample_bucket_name)
print('Object: ' + sample_bucket_object)

Bucket: my-udemy-project-210404-datalab-samples
Object: gs://my-udemy-project-210404-datalab-samples/tmp/cars.csv


In [17]:
sample_bucket = storage.Bucket(sample_bucket_name)
sample_bucket.create()
sample_bucket.exists()

True

In [18]:
table = bq.Table('importingsample.cars')
table.extract(destination = sample_bucket_object)

Job my-udemy-project-210404/job_ka1gJmJoRTPlA8K9zuenvyry0QHT completed

In [19]:
%gcs list --objects gs://$sample_bucket_name/*

Name,Type,Size,Updated
tmp/cars.csv,application/octet-stream,286,2018-08-07 04:28:19.252000+00:00


In [21]:
bucket = storage.Bucket(sample_bucket_name)

In [25]:
obj = list(bucket.objects())[0]

In [26]:
data = obj.read_stream()

In [27]:
print(data)

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000
1999,Chevy,Venture Extended Edition,"",4900
1999,Chevy,Venture Extended Edition,Very Large,5000
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799
2010,Honda,Civic,"",15000
2015,Tesla,Model S,"",64900



## Exporting Data to a Local File

In [29]:
table.to_file('/tmp/cars.csv')

In [30]:
%%bash
ls -l /tmp/cars.csv

-rw-r--r-- 1 root root 299 Aug  7 04:51 /tmp/cars.csv


In [31]:
lines = None
with open('/tmp/cars.csv') as datafile:
  lines = datafile.readlines()
print(''.join(lines))

Year,Make,Model,Description,Price
1997,Ford,E350,"ac, abs, moon",3000.0
1999,Chevy,Venture Extended Edition,,4900.0
1999,Chevy,Venture Extended Edition,Very Large,5000.0
1996,Jeep,Grand Cherokee,"MUST SELL! air, moon roof, loaded",4799.0
2010,Honda,Civic,,15000.0
2015,Tesla,Model S,,64900.0



In [45]:
%%bash
gcloud compute scp --recurse \
  datalab@training-data-analyst:/mnt/disks/datalab-pd/content/datalab/notebooks \
  /Users/april.xu/Slalom/Datalab --zone us-west1-a
  

/Users/april.xu/Slalom/Datalab: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].


In [46]:
%%bash
gcloud compute scp --recurse \
  xyingnan@training-data-analyst:/Users/april.xu/Slalom/Datalab --zone us-west1-a

ERROR: (gcloud.compute.scp) argument [[USER@]INSTANCE:]DEST: Must be specified.
Usage: gcloud compute scp [[USER@]INSTANCE:]SRC [[[USER@]INSTANCE:]SRC ...] [[USER@]INSTANCE:]DEST [optional flags]
  optional flags may be  --compress | --dry-run | --force-key-file-overwrite |
                         --help | --plain | --port | --recurse | --scp-flag |
                         --ssh-key-file | --strict-host-key-checking | --zone

For detailed information on this command and its flags, run:
  gcloud compute scp --help


In [58]:
%%bash
gcloud compute scp --recurse datalab@training-data-analyst:/content/datalab /Users/april.xu/Slalom/Datalab --zone us-west1-a

scp: /content/datalab: No such file or directory
ERROR: (gcloud.compute.scp) [/usr/bin/scp] exited with return code [1].


In [56]:
%%bash
gsutil cp *.txt gs://my-bucket

/content/datalab/docs/tutorials/BigQuery


# Cleanup

In [32]:
sample_bucket.object('tmp/cars.csv').delete()
sample_bucket.delete()
bq.Dataset('importingsample').delete(delete_contents = True)